In [1]:
root_path = "/tf/home/sergio/Tesis"

In [2]:
import sys
sys.path.append(root_path+"/TinyYOLOv3-Pedestrian-Detection")

from YOLOfunctional import TinyYOLOv3_functional,nms_layer
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
#from tensorflow.python.tools import freeze_graph
#from skimage.io import imread,imshow
#from skimage.transform import resize 
import time
#from tensorflow.compat.v1.image import decode_image
import imgaug.augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage
import os

In [3]:
#from tensorflow.keras.mixed_precision import experimental as mixed_precision

#policy = mixed_precision.Policy('mixed_float16')
#mixed_precision.set_policy(policy)

In [4]:
#physical_devices = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [5]:
#from tensorflow.keras.mixed_precision import experimental as mixed_precision

#policy = mixed_precision.Policy('mixed_float16')
#mixed_precision.set_policy(policy)

## Dataset preparation

In [6]:
image_feature_description = {
    'bboxes': tf.io.FixedLenFeature([], tf.string),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
    'num_real_boxes':tf.io.FixedLenFeature([], tf.int64),
}

In [7]:
def get_iou_matrix_tf(box_arr1, box_arr2):
    
    box_arr1 = box_arr1 -tf.tile(box_arr1[:,:2],[1,2])
    #print(box_arr1)
    x11, y11, x12, y12 = tf.split(box_arr1, 4, axis=1)
    x21, y21, x22, y22 = tf.split(box_arr2, 4, axis=1)
    xA = tf.maximum(x11, tf.transpose(x21))
    yA = tf.maximum(y11, tf.transpose(y21))
    xB = tf.minimum(x12, tf.transpose(x22))
    yB = tf.minimum(y12, tf.transpose(y22))
    interArea = tf.maximum((xB - xA + 1e-9), 0) * tf.maximum((yB - yA + 1e-9), 0)
    boxAArea = (x12 - x11 + 1e-9) * (y12 - y11 + 1e-9)
    boxBArea = (x22 - x21 + 1e-9) * (y22 - y21 + 1e-9)
    iou = interArea / (boxAArea + tf.transpose(boxBArea) - interArea)
    return iou,tf.argmax(iou,axis=1)#[:,tf.newaxis]


def fill_yolo_output(boxes,grid_size,num_anchors,which_anchor_box,which_anchor_box_index):
    #print(boxes.shape)
    #noobj_mask = tf.ones((1,grid_size*grid_size*num_anchors))
    #print(noobj_mask.shape)
    
    x_min,y_min,x_max,y_max =tf.split(boxes,4,axis=1)

    #Transforma las coordenadas de (xmin,ymin,xmax,ymax) --> (xcenter,ycenter,width,height)
    width = x_max-x_min
    height = y_max-y_min
    x_global =x_min + tf.math.divide(x_max - x_min,2)
    y_global =y_min + tf.math.divide(y_max - y_min,2)
    
    
    x_min_anchor,y_min_anchor,x_max_anchor,y_max_anchor =tf.split(which_anchor_box,4,axis=1)
    
    width_anchor = x_max_anchor-x_min_anchor
    height_anchor = y_max_anchor-y_min_anchor
    x_global_anchor =x_min_anchor + tf.math.divide(x_max_anchor - x_min_anchor,2)
    y_global_anchor =y_min_anchor + tf.math.divide(y_max_anchor - y_min_anchor,2)   

    
    #print("el x original",x_global)
    #print("el y original",y_global)
    #print("el w original",width)
    #print("el h original",height)
    
    #porción de la imagen que hay en cada celda
    pixel_per_grid = tf.math.divide(1.,grid_size)
    #print(pixel_per_grid)
    
    #Obtenemos la coordenada de la celda donde están los boundingboxes
    offset_grid_x = x_global//pixel_per_grid
    offset_grid_y = y_global//pixel_per_grid
    
    #Obtenemos el el centro locacon referencia  al celda encontrada previamente
    x_local =tf.math.floormod(x_global,pixel_per_grid)
    y_local =tf.math.floormod(y_global,pixel_per_grid)
    #print(x_local,y_local)
    
    #Valores tx e ty del groudtruth
    tx = tf.math.log(x_local + 1e-07/(1-x_local))
    ty = tf.math.log(y_local+1e-07/(1-y_local))
    tw = tf.math.log(tf.math.divide(width+1e-07,width_anchor))
    th = tf.math.log(tf.math.divide(height+1e-07,height_anchor))
    tobj_mask = tf.ones_like(tx)
    tobj = tf.concat([tobj_mask,tobj_mask],axis=0)
    
    #tnoobj = tf.zeros_like(tx)    
    #tobj = tf.ones((grid_size*grid_size*num_anchors,1))
    #tnoobj = tf.zeros((grid_size*grid_size*num_anchors,1))
    #print("Lo que la red debe predecir",tx.numpy(),ty.numpy(),tw.numpy(),th.numpy())
    #x_global = (offset_grid_x * pixel_per_grid) + tf.math.sigmoid(tx)
    #y_global = (offset_grid_y * pixel_per_grid) + tf.math.sigmoid(ty)
    #w = width_anchor*tf.math.exp(tw)
    #h = height_anchor*tf.math.exp(th)
    #print("obtnemos el x_real",x_global)
    #print("obtenemos el y_real",y_global)
    #print("obtenemos el w real",w)
    #print("obtenemos el h real",h)
    
    #anchor_boxes_per_output = num_anchors//2

    #Residuo indica cual de los 3 anchor boxes de la coordenada es la que llevara el 1
    #Coord representa la coordenada del grid
    
    residuo = tf.math.floormod(which_anchor_box_index,num_anchors)[:,tf.newaxis]
    coord = tf.cast(num_anchors*(offset_grid_y*grid_size + offset_grid_x),dtype=tf.int64)
    
    coord_objectness = tf.cast(2*(offset_grid_y*grid_size + offset_grid_x),dtype=tf.int64)
    coord_objectness2 = coord_objectness+1
    coord_objectess_global = tf.concat([coord_objectness,coord_objectness2],axis=0)
    
    output_position = residuo+coord
    print("tipo de aoutput_positivon",output_position)
    
    print(output_position)
    
    dense_shape = grid_size*grid_size*num_anchors
    print(dense_shape)
    tx_vector = tf.sparse.reorder(tf.sparse.SparseTensor(indices=output_position, values=tx[:,0], dense_shape=[dense_shape]))
    ty_vector = tf.sparse.reorder(tf.sparse.SparseTensor(indices=output_position, values=ty[:,0], dense_shape=[dense_shape]))
    tw_vector = tf.sparse.reorder(tf.sparse.SparseTensor(indices=output_position, values=tw[:,0], dense_shape=[dense_shape]))
    th_vector = tf.sparse.reorder(tf.sparse.SparseTensor(indices=output_position, values=th[:,0], dense_shape=[dense_shape]))
    obj_mask = tf.sparse.reorder(tf.sparse.SparseTensor(indices=output_position, values=tobj_mask[:,0], dense_shape=[dense_shape]))
    objectness_vector = tf.sparse.reorder(tf.sparse.SparseTensor(indices=coord_objectess_global, values=tobj[:,0], dense_shape=[dense_shape]))
    #noobj_mask = tf.sparse.reorder(tf.sparse.SparseTensor(indices=output_position, values=tnoobj[:,0], dense_shape=[dense_shape]))
    #obj_mask =tx_vector=ty_vector=tw_vector=th_vector = tf.zeros((1,grid_size*grid_size*num_anchors))
    
    tx_vector_dense = tf.sparse.to_dense(tx_vector, default_value=0, validate_indices=False, name="Dense_tx")
    ty_vector_dense = tf.sparse.to_dense(ty_vector, default_value=0, validate_indices=False, name="Dense_ty")
    tw_vector_dense = tf.sparse.to_dense(tw_vector, default_value=0, validate_indices=False, name="Dense_tw")
    th_vector_dense = tf.sparse.to_dense(th_vector, default_value=0, validate_indices=False, name="Dense_th")
    obj_mask_dense =  tf.sparse.to_dense(obj_mask, default_value=0, validate_indices=False, name="Dense_obj")
    #noobj_mask_dense = 1-obj_mask_dense
    objectness_vector_dense =  tf.sparse.to_dense(objectness_vector, default_value=0, validate_indices=False)
    
    #noobj_mask_dense= tf.sparse.to_dense(noobj_mask, default_value=1, validate_indices=False, name="Dense_noobj")
    ##print(tx_vector.to_dense)
    #print(tf.sparse.to_dense(tx_vector, default_value=0, validate_indices=True, name=None)
    #tx_vector=tx_vector[[3,2],]
    #tx_vector[output_position[:,0]] = tx
    #print("coordenada de la salida:",output_position)
    
    #return ((tx_vector_dense,ty_vector_dense,obj_mask_dense),(tw_vector_dense,th_vector_dense,obj_mask_dense),(objectness),(objectness))
    
    return tx_vector_dense,ty_vector_dense,tw_vector_dense,th_vector_dense,obj_mask_dense,objectness_vector_dense

def build_targets(image,image_bboxes,num_real_boxes,anchor_boxes):
    
    images_bboxes_original = image_bboxes
    #Obtenemos los boduing boxes que son reales
    image_bboxes = image_bboxes[:num_real_boxes,:]
    #print("Bouding boxes de la imagen",image_bboxes)
    #Obteneos  la matriz de IoU , y el índice del anchor box que dió mejor resultado
    
    #Nprmalizamos con respecto al tamaño de la imagen y obtenemos la Iou con los anchor boxes
    image_bboxes = tf.math.divide(image_bboxes,416)
    iou_matrix,which_anchor_box_index = get_iou_matrix_tf(image_bboxes,anchor_boxes)
    
    print(which_anchor_box_index)

    anchor_boxes_per_output = len(anchor_boxes)//2
    #Indices de los bouding boxes que irian en cada salida, index_best_ yolo nos dice que bouding boxes de la imagen van a la salida YOLO1,
    #porque su mejor IoU fue con los len(anchor_boxes)//2 anchor boxes mas grandes
    index_best_yolo1 = tf.where(which_anchor_box_index>=anchor_boxes_per_output)[:,0]
    index_best_yolo2 = tf.where(which_anchor_box_index<anchor_boxes_per_output)[:,0]
    index_best_anchor_yolo1 = tf.gather(which_anchor_box_index,index_best_yolo1,axis=0)
    index_best_anchor_yolo2 = tf.gather(which_anchor_box_index,index_best_yolo2,axis=0)
    
    print(index_best_yolo1)
    print(index_best_anchor_yolo1)

    print(index_best_yolo2)
    print(index_best_anchor_yolo2)

    
    best_bboxes_yolo1 = tf.gather(image_bboxes,index_best_yolo1,axis =0)
    best_anchors_yolo1 = tf.gather(anchor_boxes,index_best_anchor_yolo1, axis =0) #LOs dos anchor boxes grandes corrsponden a YOLO1
    best_bboxes_yolo2 = tf.gather(image_bboxes,index_best_yolo2,axis =0)
    best_anchors_yolo2 = tf.gather(anchor_boxes,index_best_anchor_yolo2, axis =0) #Los dos anchor boxes pequeños corresponden a YOLO2
    
    
    if best_anchors_yolo1.shape[0] !=0:
        tx_vector_yolo1,ty_vector_yolo1,tw_vector_yolo1,th_vector_yolo1,obj_mask_yolo1,obj_vector_yolo1= fill_yolo_output(best_bboxes_yolo1,13,anchor_boxes_per_output,best_anchors_yolo1,index_best_anchor_yolo1)
    else:
        tx_vector_yolo1=ty_vector_yolo1=tw_vector_yolo1=th_vector_yolo1=obj_mask_yolo1= obj_vector_yolo1=tf.zeros((1,grid_size*grid_size*num_anchors))
        #noobj_mask_yolo1 = tf.ones((1,13*13*num_anchors))
    
    if best_anchors_yolo2.shape[0] != 0:
        tx_vector_yolo2,ty_vector_yolo2,tw_vector_yolo2,th_vector_yolo2,obj_mask_yolo2,obj_vector_yolo2 = fill_yolo_output(best_bboxes_yolo2,26,anchor_boxes_per_output,best_anchors_yolo2,index_best_anchor_yolo2)
    else:
        tx_vector_yolo2=ty_vector_yolo2=tw_vector_yolo2=th_vector_yolo2=obj_mask_yolo2 = obj_vector_yolo2=tf.zeros((1,grid_size*grid_size*num_anchors))
        #noobj_mask_yolo2 = tf.ones((1,26*26*num_anchors))
        
    tx_vector = tf.concat([tx_vector_yolo1,tx_vector_yolo2],axis=0)[:,tf.newaxis]
    ty_vector = tf.concat([ty_vector_yolo1,ty_vector_yolo2],axis=0)[:,tf.newaxis]
    tw_vector = tf.concat([tw_vector_yolo1,tw_vector_yolo2],axis=0)[:,tf.newaxis]
    th_vector = tf.concat([th_vector_yolo1,th_vector_yolo2],axis=0)[:,tf.newaxis]
    obj_mask = tf.concat([obj_mask_yolo1,obj_mask_yolo2],axis=0)[:,tf.newaxis]
    #noobj_mask = tf.concat([noobj_mask_yolo1,noobj_mask_yolo2],axis=0)[:,tf.newaxis]
    obj_vector = tf.concat([obj_vector_yolo1,obj_vector_yolo2],axis=0)[:,tf.newaxis]
    
    #output = tf.concat([tx_vector,ty_vector,tw_vector,th_vector,obj_mask,noobj_mask,obj_vector],axis=1)
    #images_bboxes_original
    #return image,output
    #Vamos a regresar obj mask que es 1 cuando hay objeto en grid y el anchor box especifico
    return tf.cast(image,tf.float32)/255,(tf.concat([tx_vector,ty_vector,obj_mask],axis=1),tf.concat([tw_vector,th_vector,obj_mask],axis=1),(obj_mask),(obj_mask))

def imgaug_data_augmentation(image,bboxes,num_real_boxes):
    im_shape = image.shape
    bbs = BoundingBoxesOnImage.from_xyxy_array(bboxes*416, shape=(416,416))
    
    policy = np.random.randint(5)
    
    #policy = 2
    if policy == 0:
        
        p = np.random.random()
        if p<=0.6:
            aug = iaa.TranslateX(px=(-60, 60),cval=128)
            image, bbs = aug(image=image, bounding_boxes=bbs)
            #bbs.remove_out_of_image().clip_out_of_image()
    
        p = np.random.random()
        if p<=0.8:
            aug = iaa.HistogramEqualization()
            image, bbs = aug(image=image, bounding_boxes=bbs)
            #bbs.remove_out_of_image().clip_out_of_image()
    
    elif policy==1:
        
        p=np.random.random()
        if p<=0.2:
            aug = iaa.TranslateY(px=(int(-0.18*416), int(0.18*416)),cval=128)
            for i in bbs.to_xyxy_array(np.int32)[:num_real_boxes,:]:
                bbox = image[i[1]:i[3],i[0]:i[2]]
                bbox_augmented = aug(image=bbox)
                image[i[1]:i[3],i[0]:i[2]] = bbox_augmented
        
        p=np.random.random()
        if p<=0.8:
            square_size = np.random.randint(48)
            aug = iaa.Cutout(nb_iterations=1, size=square_size/416, squared=True)
            image, bbs = aug(image=image, bounding_boxes=bbs)
            #bbs.remove_out_of_image().clip_out_of_image()
            
    elif policy==2:
        p=np.random.random()
        if p<=1:
            aug = iaa.ShearY(shear=(int(-0.06*416), int(0.06*416)), order=1, cval=128)
            image, bbs = aug(image=image, bounding_boxes=bbs)
            #bbs.remove_out_of_image().clip_out_of_image()
            
        p=np.random.random()
        if p<=0.6:
            aug = iaa.TranslateY(px=(int(-0.18*416), int(0.18*416)),cval=128)
            for i in bbs.to_xyxy_array(np.int32)[:num_real_boxes,:]:
                bbox = image[i[1]:i[3],i[0]:i[2]]
                bbox_augmented = aug(image=bbox)
                image[i[1]:i[3],i[0]:i[2]] = bbox_augmented
            
    elif policy==3:
        p=np.random.random()
        if p<=0.6:    
            aug = iaa.Rotate(rotate=(-30, 30), order=1, cval=128)
            image, bbs = aug(image=image, bounding_boxes=bbs)
            #bbs_aug.remove_out_of_image().clip_out_of_image()
        
        p=np.random.random()
        if p<=1:
            aug = iaa.MultiplySaturation((0.54, 1.54))
            image, bbs = aug(image=image, bounding_boxes=bbs)
            #bbs.remove_out_of_image().clip_out_of_image()
            
    bbs.remove_out_of_image()
    
    return image,np.clip(bbs.to_xyxy_array(np.float32),1,415),num_real_boxes
    
    
def preprocessing(example_proto):
    image_features = tf.io.parse_single_example(example_proto, image_feature_description)
    image = tf.image.decode_jpeg(image_features['image_raw'],channels = 3)
    image = tf.cast(tf.image.resize(image,size=(416,416)), tf.uint8)
    bboxes =  tf.io.parse_tensor(image_features['bboxes'], out_type=tf.float32)
    
    num_real_boxes = image_features['num_real_boxes']
    return image,bboxes,num_real_boxes

def preprocessing_validation_set(example_proto):
    image_features = tf.io.parse_single_example(example_proto, image_feature_description)
    image = tf.image.decode_jpeg(image_features['image_raw'],channels = 3)
    image = tf.cast(tf.image.resize(image,size=(416,416)), tf.uint8)
    bboxes =  tf.io.parse_tensor(image_features['bboxes'], out_type=tf.float32)
    bboxes = tf.clip_by_value(bboxes*416,1,415)
    
    num_real_boxes = image_features['num_real_boxes']
    return image,bboxes,tf.cast(num_real_boxes,tf.int64)
    
@tf.function(input_signature=[tf.TensorSpec((416,416,3), tf.uint8),tf.TensorSpec((None,4), tf.float32),tf.TensorSpec((), tf.int64)]) 
def tf_numpy_albumentations_real(image,bboxes,num_real_boxes):
    
    boxes_shape = bboxes.shape
    im_shape = image.shape

    image,bboxes,num_real_boxes = tf.numpy_function(imgaug_data_augmentation,[image,bboxes,num_real_boxes],Tout =[tf.uint8,tf.float32,tf.int64])
 
    image.set_shape(im_shape)
    bboxes.set_shape(boxes_shape)
    print("Imagen data type",image.dtype)
    print("Bboxes data type",bboxes.dtype)
    print("num_real_boxes",num_real_boxes.dtype)

    return image,bboxes,num_real_boxes

In [8]:
#USANDO TF.IMAGE MODULE
#anchors =tf.constant(np.array([[0,0,0.015,0.037],[0,0,0.043,0.104],[0,0,0.11,0.278],[0,0,0.351,0.66]]),dtype=tf.float32)
#anchors =tf.constant(np.array([[0,0,0.026,0.062],[0,0,0.067,0.183],[0,0,0.128,0.323],[0,0,0.343,0.650]]),dtype=tf.float32)
anchors =tf.constant(np.array([[0,0,0.02078,0.049],[0,0,0.0426,0.128],[0,0,0.08523,0.19356],[0,0,0.1506,0.4163],[0,0,0.27835,0.58651],[0,0,0.5632,0.78614]]),dtype=tf.float32)



os.chdir(root_path+"/pedestrian_dataset_train_tfr")
filenames = os. listdir()
raw_image_dataset = tf.data.TFRecordDataset(filenames)

os.chdir(root_path+"/pedestrian_dataset_val_tfr_fixed")
filenames = os. listdir()
raw_image_dataset_val =tf.data.TFRecordDataset(filenames)

os.chdir(root_path+"/pedestrian_dataset_train_tfr")
#.shuffle(70000)

#VAMOS A HACER UN ENTRENAMIENTO CON DATA AUGMENTATION, ADEMAS DE USAR EL LOSS ORIGINAL

train_dataset = raw_image_dataset.map(preprocessing,num_parallel_calls=8)
train_dataset = train_dataset.map(tf_numpy_albumentations_real,num_parallel_calls=8)
train_dataset = train_dataset.map(lambda x,y,z:build_targets(x,y,z,anchors),num_parallel_calls=8)
train_dataset = train_dataset.batch(8)

val_dataset = raw_image_dataset_val.map(preprocessing_validation_set,num_parallel_calls=8)
val_dataset = val_dataset.map(lambda x,y,z:build_targets(x,y,z,anchors),num_parallel_calls=8)
val_dataset = val_dataset.batch(16)

Imagen data type <dtype: 'uint8'>
Bboxes data type <dtype: 'float32'>
num_real_boxes <dtype: 'int64'>
Tensor("ArgMax:0", shape=(None,), dtype=int64)
Tensor("strided_slice_2:0", shape=(None,), dtype=int64)
Tensor("GatherV2:0", shape=(None,), dtype=int64)
Tensor("strided_slice_3:0", shape=(None,), dtype=int64)
Tensor("GatherV2_1:0", shape=(None,), dtype=int64)
tipo de aoutput_positivon Tensor("add_18:0", shape=(None, 1), dtype=int64)
Tensor("add_18:0", shape=(None, 1), dtype=int64)
507
tipo de aoutput_positivon Tensor("add_30:0", shape=(None, 1), dtype=int64)
Tensor("add_30:0", shape=(None, 1), dtype=int64)
2028
Tensor("ArgMax:0", dtype=int64)
Tensor("strided_slice_2:0", shape=(None,), dtype=int64)
Tensor("GatherV2:0", dtype=int64)
Tensor("strided_slice_3:0", shape=(None,), dtype=int64)
Tensor("GatherV2_1:0", dtype=int64)
tipo de aoutput_positivon Tensor("add_18:0", dtype=int64)
Tensor("add_18:0", dtype=int64)
507
tipo de aoutput_positivon Tensor("add_30:0", dtype=int64)
Tensor("add_30:0

### Loss Function

In [9]:
from tensorflow.keras.losses import Loss,BinaryCrossentropy,MeanSquaredError,MeanSquaredLogarithmicError
'''
def loss_xy(y_true,y_pred):
    
    mse = MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    
    tx_true,ty_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tx_pred,ty_pred = tf.split(y_pred, [1,1], axis=-1)
    
    loss_x = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(tx_true,tx_pred)[:,:,tf.newaxis]),axis=1))
    loss_y = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(ty_true,ty_pred)[:,:,tf.newaxis]),axis=1))

    return loss_x+loss_y

def loss_wh(y_true,y_pred):
    
    mse = MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    
    tw_true,th_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tw_pred,th_pred = tf.split(y_pred, [1,1], axis=-1)
    
    loss_w = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(tw_true,tw_pred)[:,:,tf.newaxis]),axis=1))
    loss_h = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(th_true,th_pred)[:,:,tf.newaxis]),axis=1))


    return loss_w+loss_h


def loss_xy(y_true,y_pred):
    
    msle = MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.NONE)

    tx_true,ty_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tx_pred,ty_pred = tf.split(y_pred, [1,1], axis=-1)
    
    #print(tf.math.square(tx_true-tx_pred))  
    
    loss_x = tf.reduce_mean(tf.reduce_sum(obj_mask*(msle(tx_true,tx_pred)[:,:,tf.newaxis]),axis=1))
    loss_y = tf.reduce_mean(tf.reduce_sum(obj_mask*(msle(ty_true,ty_pred)[:,:,tf.newaxis]),axis=1))

    return loss_x+loss_y

def loss_wh(y_true,y_pred):

    msle = MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.NONE)
    
    tw_true,th_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tw_pred,th_pred = tf.split(y_pred, [1,1], axis=-1)
    
    #print(tf.math.square(tw_true-tw_pred))

    
    loss_w = tf.reduce_mean(tf.reduce_sum(obj_mask*(msle(tw_true,tw_pred)[:,:,tf.newaxis]),axis=1))
    loss_h = tf.reduce_mean(tf.reduce_sum(obj_mask*(msle(th_true,th_pred)[:,:,tf.newaxis]),axis=1))


    return loss_w+loss_h
    
def loss_xy(y_true,y_pred):
    
    msle = MeanSquaredLogarithmicError(reduction=tf.keras.losses.Reduction.NONE)

    tx_true,ty_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tx_pred,ty_pred = tf.split(y_pred, [1,1], axis=-1)
    
    #print(tf.math.square(tx_true-tx_pred))  
    
    loss_x = tf.reduce_mean(tf.reduce_sum(obj_mask*(msle(tx_true,tx_pred)[:,:,tf.newaxis]),axis=1))
    loss_y = tf.reduce_mean(tf.reduce_sum(obj_mask*(msle(ty_true,ty_pred)[:,:,tf.newaxis]),axis=1))

    return loss_x+loss_y
    
def loss_wh(y_true,y_pred):

    mse = MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    
    tw_true,th_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tw_pred,th_pred = tf.split(y_pred, [1,1], axis=-1)
    
    #print(tf.math.square(tw_true-tw_pred))

    
    loss_w = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(tf.math.exp(tw_true),tf.math.exp(tw_pred))[:,:,tf.newaxis]),axis=1))
    loss_h = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(tf.math.exp(th_true),tf.math.exp(th_pred))[:,:,tf.newaxis]),axis=1))


    return loss_w+loss_h
    
'''


def loss_xy(y_true,y_pred):
    
    mse = MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    
    tx_true,ty_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tx_pred,ty_pred = tf.split(y_pred, [1,1], axis=-1)
    
    loss_x = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(tx_true,tx_pred)[:,:,tf.newaxis]),axis=1))
    loss_y = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(ty_true,ty_pred)[:,:,tf.newaxis]),axis=1))

    return loss_x+loss_y




def loss_wh(y_true,y_pred):
    
    mse = MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    
    tw_true,th_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    tw_pred,th_pred = tf.split(y_pred, [1,1], axis=-1)
    
    loss_w = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(tw_true,tw_pred)[:,:,tf.newaxis]),axis=1))
    loss_h = tf.reduce_mean(tf.reduce_sum(obj_mask*(mse(th_true,th_pred)[:,:,tf.newaxis]),axis=1))


    return loss_w+loss_h

def loss_objectness(y_true,y_pred):
    bce = BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    
    #tw_true,th_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    #tw_pred,th_pred = tf.split(y_pred, [1,1], axis=-1)   
    
    loss_obj =tf.reduce_mean(tf.reduce_sum( y_true*bce(y_true,y_pred)[:,tf.newaxis],axis=1))
    
    return loss_obj

def loss_no_objectness(y_true,y_pred):
    bce = BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    
    
    loss_noobj =tf.reduce_mean(tf.reduce_sum((1-y_true)*bce(y_true,y_pred)[:,tf.newaxis],axis=1))
    
    return loss_noobj
'''
def loss_bce_objectness(y_true,y_pred):
    #bce = BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    #tw_true,th_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)
    #tw_pred,th_pred = tf.split(y_pred, [1,1], axis=-1)   
    
    loss_obj =tf.reduce_mean(tf.reduce_sum( y_true*tf.math.log(y_pred+1e-7)[:,tf.newaxis],axis=1))
    
    return loss_obj

def loss_bce_no_objectness(y_true,y_pred):
    #bce = BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    loss_noobj =tf.reduce_mean(tf.reduce_sum((1-y_true)*tf.math.log(1-y_pred+1e-7)[:,tf.newaxis],axis=1))
    
    return loss_noobj
'''

'\ndef loss_bce_objectness(y_true,y_pred):\n    #bce = BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)\n    #tw_true,th_true,obj_mask = tf.split(y_true, [1,1,1], axis=-1)\n    #tw_pred,th_pred = tf.split(y_pred, [1,1], axis=-1)   \n    \n    loss_obj =tf.reduce_mean(tf.reduce_sum( y_true*tf.math.log(y_pred+1e-7)[:,tf.newaxis],axis=1))\n    \n    return loss_obj\n\ndef loss_bce_no_objectness(y_true,y_pred):\n    #bce = BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)\n    loss_noobj =tf.reduce_mean(tf.reduce_sum((1-y_true)*tf.math.log(1-y_pred+1e-7)[:,tf.newaxis],axis=1))\n    \n    return loss_noobj\n'

## Loading the functional model

In [12]:
#anchors =[[10/416,14/416],[23/416,27/416],[37/416,58/416],[81/416,82/416],[135/416,169/416],[344/416,319/416]]
anchors =[[0.02078,0.049],[0.0426,0.128],[0.08523,0.19356],[0.1506,0.4163],[0.27835,0.58651],[0.5632,0.78614]]

model = TinyYOLOv3_functional(anchor_boxes = anchors,num_classes=0,mode="tl")
model.training = False
current_directory= '/tf/home/sergio/Tesis/TinyYOLOv3-Pedestrian-Detection/Deployment'
model.load_weights(current_directory+'/weights_functional_one_class/original_model_one_class')
#model.load_weights_darknet("/home/sergio/TinyYOLOv3-Pedestrian-Detection/yolov3-tiny.weights");

In [13]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 416, 416, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 416, 416, 16) 432         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 416, 416, 16) 64          conv2d[0][0]                     
__________________________________________________________________________________________________
tf_op_layer_LeakyRelu (TensorFl [(None, 416, 416, 16 0           batch_normalization[0][0]        
_______________________________________________________________________________________

## Training with Model.fit()

## Eager execution training (For debuging)

In [14]:
debugging = False

if debugging:
    model = TinyYOLOv3_functional(anchor_boxes = anchors,num_classes=0,mode="tl")
    model.training = False
    current_directory= '/tf/home/sergio/Tesis/TinyYOLOv3-Pedestrian-Detection/Deployment'
    model.load_weights(current_directory+'/weights_functional_one_class/original_model_one_class')
    
    for epochs in range(1,2,1):
        for (images,y_true) in train_dataset:
            with tf.GradientTape() as tape:
                outputs = model(images)
                #print("Tamaño de la etiqueta",y_true[0].shape)
                #print("Salida",outputs[0].shape)
                loss_x_y = loss_xy(y_true[0],outputs[0])
                loss_w_h = loss_wh(y_true[1],outputs[1])
                loss_obj = loss_objectness(y_true[2],outputs[2])
                loss_noobj = loss_no_objectness(y_true[3],outputs[3])

                total_loss =loss_x_y+loss_w_h+loss_obj+loss_noobj
            print(epochs,(total_loss.numpy()))
            grads = tape.gradient(total_loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

                #logging.info("{}_train_{}, {}, {}".format(
                #    epoch, batch, total_loss.numpy()))
                
else:
    pass

In [15]:
reduce_lr =tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_Concatenate_obj_recall', factor=0.2, patience=5, verbose=1, mode='auto',min_delta=0.05)


checkpoint_path =root_path+  "/training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor='val_Concatenate_obj_recall',
    mode='max',
    save_best_only=True,verbose=1)


tf.random.set_seed(0)

from tensorflow.keras.metrics import TrueNegatives,TruePositives,FalseNegatives,FalsePositives,Precision,Recall

opt = tf.keras.optimizers.Nadam(3e-4)

losses = {"tf_op_layer_split_2": loss_xy,
          "tf_op_layer_split_2_1": loss_wh,
          "Concatenate_obj":loss_objectness,
          "Concatenate_obj_1":loss_no_objectness  
}

metrics = {"Concatenate_obj":[Precision(0.5),Recall(0.5),TruePositives(0.5),FalsePositives(0.5)]}
model.compile(optimizer=opt, loss=losses,metrics=metrics,loss_weights=[4,4,2,1])
os.chdir(root_path+ "/pedestrian_dataset_train_tfr")

## Batch 16, con pesos 1,1,1,1 with lr=0.0001 y usando mse para el (x,y) y usando mse para (w,h)

In [16]:
history = model.fit(train_dataset, epochs=40,validation_data=val_dataset,callbacks=[model_checkpoint_callback])

Epoch 1/40
    255/Unknown - 8s 30ms/step - loss: 561.3698 - tf_op_layer_split_2_loss: 62.3263 - tf_op_layer_split_2_1_loss: 11.3840 - Concatenate_obj_loss: 0.6863 - Concatenate_obj_1_loss: 411.2038 - Concatenate_obj_precision_1: 0.0026 - Concatenate_obj_recall_1: 0.0947 - Concatenate_obj_true_positives_1: 731.0000 - Concatenate_obj_false_positives_1: 277822.0000

KeyboardInterrupt: 

In [15]:
current_directory= '/tf/home/sergio/Tesis/TinyYOLOv3-Pedestrian-Detection/Deployment'

In [16]:
model.save_weights(current_directory+'/weights_retrain_functional/transfer_learning_experiment1')

## Fine Tuning

In [10]:
#anchors =[[10/416,14/416],[23/416,27/416],[37/416,58/416],[81/416,82/416],[135/416,169/416],[344/416,319/416]]
anchors =[[0.02078,0.049],[0.0426,0.128],[0.08523,0.19356],[0.1506,0.4163],[0.27835,0.58651],[0.5632,0.78614]]

model = TinyYOLOv3_functional(anchor_boxes = anchors,num_classes=0,mode="ft")
model.training = True
current_directory= '/tf/home/sergio/Tesis/TinyYOLOv3-Pedestrian-Detection/Deployment'
model.load_weights(current_directory+'/weights_retrain_functional/transfer_learning_experiment1')

In [13]:
reduce_lr =tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_Concatenate_obj_recall', factor=0.2, patience=5, verbose=1, mode='auto',min_delta=0.05)


checkpoint_path =root_path+  "training_2/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor='val_Concatenate_obj_recall',
    mode='max',
    save_best_only=True,verbose=1)


tf.random.set_seed(0)

from tensorflow.keras.metrics import TrueNegatives,TruePositives,FalseNegatives,FalsePositives,Precision,Recall

opt = tf.keras.optimizers.Nadam(3e-4)

losses = {"tf_op_layer_split_2": loss_xy,
          "tf_op_layer_split_2_1": loss_wh,
          "Concatenate_obj":loss_objectness,
          "Concatenate_obj_1":loss_no_objectness  
}

metrics = {"Concatenate_obj":[Precision(0.5),Recall(0.5),TruePositives(0.5),FalsePositives(0.5)]}
model.compile(optimizer=opt, loss=losses,metrics=metrics,loss_weights=[4,4,2,1])
os.chdir(root_path+ "/pedestrian_dataset_train_tfr")

In [14]:
os.chdir(root_path+ "/pedestrian_dataset_train_tfr")
history = model.fit(train_dataset, epochs=20,validation_data=val_dataset,callbacks=[model_checkpoint_callback])

Epoch 1/20
   9213/Unknown - 701s 76ms/step - loss: 46.6265 - tf_op_layer_split_2_loss: 5.7335 - tf_op_layer_split_2_1_loss: 2.1941 - Concatenate_obj_loss: 0.0431 - Concatenate_obj_1_loss: 14.8300 - Concatenate_obj_precision_2: 0.6062 - Concatenate_obj_recall_2: 0.0603 - Concatenate_obj_true_positives_2: 16309.0000 - Concatenate_obj_false_positives_2: 10594.0000WARNING:tensorflow:Can save best model only with val_Concatenate_obj_recall available, skipping.
9213/9213 [==============================] - 708s 77ms/step - loss: 46.6265 - tf_op_layer_split_2_loss: 5.7335 - tf_op_layer_split_2_1_loss: 2.1941 - Concatenate_obj_loss: 0.0431 - Concatenate_obj_1_loss: 14.8300 - Concatenate_obj_precision_2: 0.6062 - Concatenate_obj_recall_2: 0.0603 - Concatenate_obj_true_positives_2: 16309.0000 - Concatenate_obj_false_positives_2: 10594.0000 - val_loss: 40.2709 - val_tf_op_layer_split_2_loss: 5.2061 - val_tf_op_layer_split_2_1_loss: 0.7740 - val_Concatenate_obj_loss: 0.0510 - val_Concatenate_obj_1

Epoch 8/20
9213/9213 [==============================] - 706s 77ms/step - loss: 37.1293 - tf_op_layer_split_2_loss: 4.5486 - tf_op_layer_split_2_1_loss: 1.4401 - Concatenate_obj_loss: 0.0383 - Concatenate_obj_1_loss: 13.0977 - Concatenate_obj_precision_2: 0.6468 - Concatenate_obj_recall_2: 0.1170 - Concatenate_obj_true_positives_2: 31607.0000 - Concatenate_obj_false_positives_2: 17260.0000 - val_loss: 32.3938 - val_tf_op_layer_split_2_loss: 3.8683 - val_tf_op_layer_split_2_1_loss: 0.5681 - val_Concatenate_obj_loss: 0.0460 - val_Concatenate_obj_1_loss: 14.5565 - val_Concatenate_obj_precision_2: 0.7512 - val_Concatenate_obj_recall_2: 0.0687 - val_Concatenate_obj_true_positives_2: 812.0000 - val_Concatenate_obj_false_positives_2: 269.0000
Epoch 9/20
9213/9213 [==============================] - 700s 76ms/step - loss: 36.6993 - tf_op_layer_split_2_loss: 4.4385 - tf_op_layer_split_2_1_loss: 1.4607 - Concatenate_obj_loss: 0.0381 - Concatenate_obj_1_loss: 13.0265 - Concatenate_obj_precision_2: 

9213/9213 [==============================] - 703s 76ms/step - loss: 34.0252 - tf_op_layer_split_2_loss: 4.0240 - tf_op_layer_split_2_1_loss: 1.3191 - Concatenate_obj_loss: 0.0369 - Concatenate_obj_1_loss: 12.5791 - Concatenate_obj_precision_2: 0.6569 - Concatenate_obj_recall_2: 0.1370 - Concatenate_obj_true_positives_2: 37028.0000 - Concatenate_obj_false_positives_2: 19344.0000 - val_loss: 28.9101 - val_tf_op_layer_split_2_loss: 3.1254 - val_tf_op_layer_split_2_1_loss: 0.6756 - val_Concatenate_obj_loss: 0.0428 - val_Concatenate_obj_1_loss: 13.6203 - val_Concatenate_obj_precision_2: 0.7672 - val_Concatenate_obj_recall_2: 0.0886 - val_Concatenate_obj_true_positives_2: 1048.0000 - val_Concatenate_obj_false_positives_2: 318.0000
Epoch 15/20
9213/9213 [==============================] - 702s 76ms/step - loss: 33.4235 - tf_op_layer_split_2_loss: 3.8918 - tf_op_layer_split_2_1_loss: 1.3184 - Concatenate_obj_loss: 0.0367 - Concatenate_obj_1_loss: 12.5093 - Concatenate_obj_precision_2: 0.6602 - 

In [13]:
current_directory= '/tf/home/sergio/Tesis/TinyYOLOv3-Pedestrian-Detection/Deployment'

In [14]:
model.save_weights(current_directory+'/weights_retrain_functional/fine_tuning_part_1_experiment1')

## CONTINUACIÓN

In [9]:
#anchors =[[10/416,14/416],[23/416,27/416],[37/416,58/416],[81/416,82/416],[135/416,169/416],[344/416,319/416]]
anchors =[[0.02078,0.049],[0.0426,0.128],[0.08523,0.19356],[0.1506,0.4163],[0.27835,0.58651],[0.5632,0.78614]]

model = TinyYOLOv3_functional(anchor_boxes = anchors,num_classes=0,mode="ft")
model.training = False
current_directory= '/tf/home/sergio/Tesis/TinyYOLOv3-Pedestrian-Detection/Deployment'
model.load_weights(current_directory+'/weights_retrain_functional/fine_tuning_part_1_experiment1')

In [10]:
reduce_lr =tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_Concatenate_obj_recall', factor=0.1, patience=4, verbose=1, mode='auto',min_delta=0.05)


checkpoint_path = root_path+ "/training_3/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor='val_Concatenate_obj_recall',
    mode='max',
    save_best_only=True,verbose=1)


tf.random.set_seed(0)

from tensorflow.keras.metrics import TrueNegatives,TruePositives,FalseNegatives,FalsePositives,Precision,Recall

opt = tf.keras.optimizers.Adam(3e-4)

losses = {"tf_op_layer_split_2": loss_xy,
          "tf_op_layer_split_2_1": loss_wh,
          "Concatenate_obj":loss_objectness,
          "Concatenate_obj_1":loss_no_objectness  
}

metrics = {"Concatenate_obj":[Precision(0.5),Recall(0.5),TruePositives(0.5),FalsePositives(0.5)]}
model.compile(optimizer=opt, loss=losses,metrics=metrics,loss_weights=[2,1,2,1])
os.chdir(root_path+ "/pedestrian_dataset_train_tfr")

In [11]:
os.chdir(root_path+ "/pedestrian_dataset_train_tfr")
history = model.fit(train_dataset, epochs=40,validation_data=val_dataset,callbacks=[model_checkpoint_callback])

Epoch 1/40
   9213/Unknown - 661s 72ms/step - loss: 24.7276 - tf_op_layer_split_2_loss: 5.0535 - tf_op_layer_split_2_1_loss: 1.6996 - Concatenate_obj_loss: 0.0378 - Concatenate_obj_1_loss: 12.8455 - Concatenate_obj_precision: 0.6334 - Concatenate_obj_recall: 0.1161 - Concatenate_obj_true_positives: 31362.0000 - Concatenate_obj_false_positives: 18154.0000- 653s 72ms/step - loss: 24.8774 - tf_op_layer_split_2_loss: 5.0855 - tf_op_layer_split_2_1_loss: 1.7171 - Concatenate_obj_loss: 0.0382 - Concatenate_obj_1_loss: 12.9129 - Concatenate_obj_precision: 0.6350 - Concatenate
Epoch 00001: val_Concatenate_obj_recall improved from -inf to 0.13735, saving model to /tf/home/sergio/Tesis/training_3/cp.ckpt
9213/9213 [==============================] - 668s 72ms/step - loss: 24.7276 - tf_op_layer_split_2_loss: 5.0535 - tf_op_layer_split_2_1_loss: 1.6996 - Concatenate_obj_loss: 0.0378 - Concatenate_obj_1_loss: 12.8455 - Concatenate_obj_precision: 0.6334 - Concatenate_obj_recall: 0.1161 - Concatenate_

9212/9213 [============================>.] - ETA: 0s - loss: 24.1793 - tf_op_layer_split_2_loss: 4.8928 - tf_op_layer_split_2_1_loss: 1.6791 - Concatenate_obj_loss: 0.0373 - Concatenate_obj_1_loss: 12.6399 - Concatenate_obj_precision: 0.6429 - Concatenate_obj_recall: 0.1249 - Concatenate_obj_true_positives: 33761.0000 - Concatenate_obj_false_positives: 18750.0000- ETA: 9s - loss: 24.3488 - tf_op_layer_split_2_loss: 4.9296 - tf_op_layer_split_2_1_loss: 1.6998 - Concatenate_obj_loss: 0.0377 - Concatenate_obj_1_loss: 12.7143 - Concatenate_obj_precision: 0.6445 - Concatenate_o - ETA: 1s - loss: 24.2080 - tf_op_layer_split_2_loss: 4.8990 - tf_op_layer_split_2_1_loss: 1.6826 - Concatenate_obj_loss: 0.0374 - Concatenate_obj_1_loss: 12.6526 - Concatenate_obj_precision: 0.6432 - Concatenate_obj_recall: 0.1250 - Concatenate_obj_true_positives: 33730.0000 - Concatenate_obj_false_p
Epoch 00013: val_Concatenate_obj_recall improved from 0.14454 to 0.14657, saving model to /tf/home/sergio/Tesis/train

Epoch 26/40
9212/9213 [============================>.] - ETA: 0s - loss: 23.7289 - tf_op_layer_split_2_loss: 4.7914 - tf_op_layer_split_2_1_loss: 1.6428 - Concatenate_obj_loss: 0.0367 - Concatenate_obj_1_loss: 12.4298 - Concatenate_obj_precision: 0.6462 - Concatenate_obj_recall: 0.1321 - Concatenate_obj_true_positives: 35682.0000 - Concatenate_obj_false_positives: 19539.0000
Epoch 00026: val_Concatenate_obj_recall improved from 0.15054 to 0.15122, saving model to /tf/home/sergio/Tesis/training_3/cp.ckpt
9213/9213 [==============================] - 667s 72ms/step - loss: 23.7286 - tf_op_layer_split_2_loss: 4.7913 - tf_op_layer_split_2_1_loss: 1.6428 - Concatenate_obj_loss: 0.0367 - Concatenate_obj_1_loss: 12.4297 - Concatenate_obj_precision: 0.6462 - Concatenate_obj_recall: 0.1321 - Concatenate_obj_true_positives: 35682.0000 - Concatenate_obj_false_positives: 19539.0000 - val_loss: 21.5787 - val_tf_op_layer_split_2_loss: 4.4671 - val_tf_op_layer_split_2_1_loss: 0.6399 - val_Concatenate_

Epoch 33/40
9212/9213 [============================>.] - ETA: 0s - loss: 23.4599 - tf_op_layer_split_2_loss: 4.7282 - tf_op_layer_split_2_1_loss: 1.5687 - Concatenate_obj_loss: 0.0366 - Concatenate_obj_1_loss: 12.3616 - Concatenate_obj_precision: 0.6446 - Concatenate_obj_recall: 0.1357 - Concatenate_obj_true_positives: 36678.0000 - Concatenate_obj_false_positives: 20225.0000
Epoch 00033: val_Concatenate_obj_recall did not improve from 0.15384
9213/9213 [==============================] - 668s 73ms/step - loss: 23.4597 - tf_op_layer_split_2_loss: 4.7281 - tf_op_layer_split_2_1_loss: 1.5686 - Concatenate_obj_loss: 0.0366 - Concatenate_obj_1_loss: 12.3616 - Concatenate_obj_precision: 0.6446 - Concatenate_obj_recall: 0.1357 - Concatenate_obj_true_positives: 36679.0000 - Concatenate_obj_false_positives: 20225.0000 - val_loss: 21.3432 - val_tf_op_layer_split_2_loss: 4.3948 - val_tf_op_layer_split_2_1_loss: 0.6203 - val_Concatenate_obj_loss: 0.0365 - val_Concatenate_obj_1_loss: 11.8602 - val_C

Epoch 40/40
9212/9213 [============================>.] - ETA: 0s - loss: 23.1874 - tf_op_layer_split_2_loss: 4.6814 - tf_op_layer_split_2_1_loss: 1.4923 - Concatenate_obj_loss: 0.0363 - Concatenate_obj_1_loss: 12.2597 - Concatenate_obj_precision: 0.6489 - Concatenate_obj_recall: 0.1393 - Concatenate_obj_true_positives: 37629.0000 - Concatenate_obj_false_positives: 20364.0000- ETA: 2s - loss: 23.2352 - tf_op_layer_split_2_loss: 4.6915 - tf_op_layer_split_2_1_loss: 1.4976 - Concatenate_obj_loss: 0.0364 - Concatenate_obj_1_loss: 12.2816 - Concatenate_obj_precision: 0.6494 - Concatenate_obj_recall: 0.1393 - Concatenate_obj_true_positives: 37560.0000 - Concaten
Epoch 00040: val_Concatenate_obj_recall did not improve from 0.15663
9213/9213 [==============================] - 682s 74ms/step - loss: 23.1872 - tf_op_layer_split_2_loss: 4.6814 - tf_op_layer_split_2_1_loss: 1.4922 - Concatenate_obj_loss: 0.0363 - Concatenate_obj_1_loss: 12.2596 - Concatenate_obj_precision: 0.6489 - Concatenate_obj

In [12]:
current_directory= '/tf/home/sergio/Tesis/TinyYOLOv3-Pedestrian-Detection/Deployment'

In [13]:
model.save_weights(current_directory+'/weights_retrain_functional/fine_tuning_part_2_experiment1')

## Continuacion

In [9]:
#anchors =[[10/416,14/416],[23/416,27/416],[37/416,58/416],[81/416,82/416],[135/416,169/416],[344/416,319/416]]
anchors =[[0.02078,0.049],[0.0426,0.128],[0.08523,0.19356],[0.1506,0.4163],[0.27835,0.58651],[0.5632,0.78614]]

model = TinyYOLOv3_functional(anchor_boxes = anchors,num_classes=0,mode="tl",training=True)
model.training = False
current_directory= '/tf/home/sergio/Tesis/TinyYOLOv3-Pedestrian-Detection/Deployment'
model.load_weights("/tf/home/sergio/Tesis/training_3/cp.ckpt")

In [10]:
reduce_lr =tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_Concatenate_obj_recall', factor=0.1, patience=4, verbose=1, mode='auto',min_delta=0.05)


checkpoint_path = root_path+ "/training_3/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor='val_Concatenate_obj_recall',
    mode='max',
    save_best_only=True,verbose=1)


tf.random.set_seed(0)

from tensorflow.keras.metrics import TrueNegatives,TruePositives,FalseNegatives,FalsePositives,Precision,Recall

opt = tf.keras.optimizers.Adam()

losses = {"tf_op_layer_split_2": loss_xy,
          "tf_op_layer_split_2_1": loss_wh,
          "Concatenate_obj":loss_objectness,
          "Concatenate_obj_1":loss_no_objectness  
}

metrics = {"Concatenate_obj":[Precision(0.5),Recall(0.5),TruePositives(0.5),FalsePositives(0.5)]}
model.compile(optimizer=opt, loss=losses,metrics=metrics,loss_weights=[2,1,2,1])
os.chdir(root_path+ "/pedestrian_dataset_train_tfr")

In [11]:
os.chdir(root_path+ "/pedestrian_dataset_train_tfr")
history = model.fit(train_dataset, epochs=20,validation_data=val_dataset,callbacks=[model_checkpoint_callback])

Epoch 1/20
   9213/Unknown - 309s 34ms/step - loss: 23.1843 - tf_op_layer_split_2_loss: 4.7072 - tf_op_layer_split_2_1_loss: 1.5619 - Concatenate_obj_loss: 0.0359 - Concatenate_obj_1_loss: 12.1363 - Concatenate_obj_precision: 0.6566 - Concatenate_obj_recall: 0.1352 - Concatenate_obj_true_positives: 36536.0000 - Concatenate_obj_false_positives: 19108.0000
Epoch 00001: val_Concatenate_obj_recall improved from -inf to 0.16788, saving model to /tf/home/sergio/Tesis/training_3/cp.ckpt
9213/9213 [==============================] - 317s 34ms/step - loss: 23.1843 - tf_op_layer_split_2_loss: 4.7072 - tf_op_layer_split_2_1_loss: 1.5619 - Concatenate_obj_loss: 0.0359 - Concatenate_obj_1_loss: 12.1363 - Concatenate_obj_precision: 0.6566 - Concatenate_obj_recall: 0.1352 - Concatenate_obj_true_positives: 36536.0000 - Concatenate_obj_false_positives: 19108.0000 - val_loss: 21.1577 - val_tf_op_layer_split_2_loss: 4.3489 - val_tf_op_layer_split_2_1_loss: 0.6142 - val_Concatenate_obj_loss: 0.0362 - val_C

9212/9213 [============================>.] - ETA: 0s - loss: 23.2337 - tf_op_layer_split_2_loss: 4.7443 - tf_op_layer_split_2_1_loss: 1.5563 - Concatenate_obj_loss: 0.0358 - Concatenate_obj_1_loss: 12.1170 - Concatenate_obj_precision: 0.6453 - Concatenate_obj_recall: 0.1449 - Concatenate_obj_true_positives: 39137.0000 - Concatenate_obj_false_positives: 21508.0000
Epoch 00013: val_Concatenate_obj_recall did not improve from 0.17718
9213/9213 [==============================] - 270s 29ms/step - loss: 23.2334 - tf_op_layer_split_2_loss: 4.7443 - tf_op_layer_split_2_1_loss: 1.5563 - Concatenate_obj_loss: 0.0358 - Concatenate_obj_1_loss: 12.1170 - Concatenate_obj_precision: 0.6453 - Concatenate_obj_recall: 0.1449 - Concatenate_obj_true_positives: 39137.0000 - Concatenate_obj_false_positives: 21510.0000 - val_loss: 21.1134 - val_tf_op_layer_split_2_loss: 4.3392 - val_tf_op_layer_split_2_1_loss: 0.6107 - val_Concatenate_obj_loss: 0.0361 - val_Concatenate_obj_1_loss: 11.7521 - val_Concatenate_o

Epoch 20/20
9213/9213 [==============================] - ETA: 0s - loss: 23.1517 - tf_op_layer_split_2_loss: 4.7371 - tf_op_layer_split_2_1_loss: 1.5063 - Concatenate_obj_loss: 0.0357 - Concatenate_obj_1_loss: 12.0997 - Concatenate_obj_precision: 0.6431 - Concatenate_obj_recall: 0.1459 - Concatenate_obj_true_positives: 39404.0000 - Concatenate_obj_false_positives: 21864.0000
Epoch 00020: val_Concatenate_obj_recall improved from 0.17845 to 0.17854, saving model to /tf/home/sergio/Tesis/training_3/cp.ckpt
9213/9213 [==============================] - 283s 31ms/step - loss: 23.1517 - tf_op_layer_split_2_loss: 4.7371 - tf_op_layer_split_2_1_loss: 1.5063 - Concatenate_obj_loss: 0.0357 - Concatenate_obj_1_loss: 12.0997 - Concatenate_obj_precision: 0.6431 - Concatenate_obj_recall: 0.1459 - Concatenate_obj_true_positives: 39404.0000 - Concatenate_obj_false_positives: 21864.0000 - val_loss: 21.1037 - val_tf_op_layer_split_2_loss: 4.3364 - val_tf_op_layer_split_2_1_loss: 0.6095 - val_Concatenate_

In [12]:
current_directory= '/tf/home/sergio/Tesis/TinyYOLOv3-Pedestrian-Detection/Deployment'

In [13]:
model.save_weights(current_directory+'/weights_retrain_functional/fine_tuning_part_3_experiment1')

## COntinuacion Fine Tuning FInal

In [11]:
#anchors =[[10/416,14/416],[23/416,27/416],[37/416,58/416],[81/416,82/416],[135/416,169/416],[344/416,319/416]]
anchors =[[0.02078,0.049],[0.0426,0.128],[0.08523,0.19356],[0.1506,0.4163],[0.27835,0.58651],[0.5632,0.78614]]

model = TinyYOLOv3_functional(anchor_boxes = anchors,num_classes=0,mode="ft",training=True)
model.training = False
current_directory= '/tf/home/sergio/Tesis/TinyYOLOv3-Pedestrian-Detection/Deployment'
model.load_weights("/tf/home/sergio/Tesis/training_3/cp.ckpt")

In [10]:
reduce_lr =tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_Concatenate_obj_recall', factor=0.1, patience=4, verbose=1, mode='auto',min_delta=0.05)


checkpoint_path = root_path+ "/training_4/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    monitor='val_Concatenate_obj_recall',
    mode='max',
    save_best_only=True,verbose=1)


tf.random.set_seed(0)

from tensorflow.keras.metrics import TrueNegatives,TruePositives,FalseNegatives,FalsePositives,Precision,Recall

opt = tf.keras.optimizers.Adam(1e-5)

losses = {"tf_op_layer_split_2": loss_xy,
          "tf_op_layer_split_2_1": loss_wh,
          "Concatenate_obj":loss_objectness,
          "Concatenate_obj_1":loss_no_objectness  
}

metrics = {"Concatenate_obj":[Precision(0.5),Recall(0.5),TruePositives(0.5),FalsePositives(0.5)]}
model.compile(optimizer=opt, loss=losses,metrics=metrics,loss_weights=[2,1,2,1])
os.chdir(root_path+ "/pedestrian_dataset_train_tfr")

In [11]:
os.chdir(root_path+ "/pedestrian_dataset_train_tfr")
history = model.fit(train_dataset, epochs=20,validation_data=val_dataset,callbacks=[model_checkpoint_callback])

Epoch 1/20
   9213/Unknown - 657s 71ms/step - loss: 23.3202 - tf_op_layer_split_2_loss: 4.6950 - tf_op_layer_split_2_1_loss: 1.5623 - Concatenate_obj_loss: 0.0364 - Concatenate_obj_1_loss: 12.2951 - Concatenate_obj_precision: 0.6463 - Concatenate_obj_recall: 0.1410 - Concatenate_obj_true_positives: 38108.0000 - Concatenate_obj_false_positives: 20858.0000- 615s 71ms/step - loss: 23.3505 - tf_op_layer_split_2_loss
Epoch 00001: val_Concatenate_obj_recall improved from -inf to 0.15680, saving model to /tf/home/sergio/Tesis/training_4/cp.ckpt
9213/9213 [==============================] - 665s 72ms/step - loss: 23.3202 - tf_op_layer_split_2_loss: 4.6950 - tf_op_layer_split_2_1_loss: 1.5623 - Concatenate_obj_loss: 0.0364 - Concatenate_obj_1_loss: 12.2951 - Concatenate_obj_precision: 0.6463 - Concatenate_obj_recall: 0.1410 - Concatenate_obj_true_positives: 38108.0000 - Concatenate_obj_false_positives: 20858.0000 - val_loss: 21.1535 - val_tf_op_layer_split_2_loss: 4.3429 - val_tf_op_layer_split_

KeyboardInterrupt: 